In [16]:
import pandas as pd
import numpy as np
import torch
import monai
import torchvision
import numpy as np
import os
import shutil
import matplotlib
from PIL import Image
from tqdm import tqdm
import os
from glob import glob
from torch.utils.data import DataLoader, Dataset
from monai.transforms import (
    Compose,
    Resized,
    ToTensord,
    LoadImaged,
    MapTransform,
    NormalizeIntensityd,
    Orientationd,
    RandFlipd,
    RandShiftIntensityd,
    Spacingd,
    EnsureChannelFirstd,
    RandSpatialCropd,
    RandScaleIntensityd
)



In [22]:
import os
import nibabel as nib
import torch
from torch.utils.data import Dataset
import torchio as tio

class imagedata(Dataset):
    def __init__(self, img_dir, label_dir, transform=None):
        self.img_dir = img_dir
        self.label_dir = label_dir
        self.transform = transform
        self.patients_list = sorted(set('_'.join(f.split('_')[:-1]) for f in os.listdir(img_dir) if f.endswith('.nii')))

    def __getitem__(self, index):
        patient_id = self.patients_list[index]
        modalities = ['t1', 'flair', 't2']
        images = {mod: self.load_nifti_image(f"{patient_id}_{mod}.nii") for mod in modalities}
        mask = self.load_nifti_image(f"{patient_id}_seg.nii")

        # Create torchio Images
        images = {mod: tio.ScalarImage(tensor=image) for mod, image in images.items()}
        mask = tio.LabelMap(tensor=mask)

        # Create torchio Subject
        sample = tio.Subject(images, mask=mask)

        if self.transform:
            sample = self.transform(sample)

        # Ensure the images and mask have the correct shape
        for key in sample:
            sample[key] = self.adjust_dimensions(sample[key].data)
        
        images = {mod: sample[mod] for mod in modalities}
        mask = sample['mask']
        
        return images, mask

    def __len__(self):
        return len(self.patients_list)

    def load_nifti_image(self, filename):
        filepath = os.path.join(self.img_dir, filename) if 'seg' not in filename else os.path.join(self.label_dir, filename)
        image = nib.load(filepath).get_fdata()
        return torch.from_numpy(image).unsqueeze(0).float()

    def adjust_dimensions(self, tensor):
        # Permute dimensions to match (in_channels, D, H, W)
        return tensor.permute(0, 3, 1, 2)  # From (1, H, W, D) to (1, D, H, W)

# Define your transformations
data_transforms = tio.Compose([
    tio.RescaleIntensity((0, 1)),  # Normalize intensity to [0, 1]
    tio.Resample((1.0, 1.0, 1.0)),  # Resample to a fixed voxel size if needed
    tio.Resize((128, 128, 128)),  # Resize images to 128x128x128
    tio.RandomFlip(axes=(0, 1, 2)),  # Random flip along spatial axes
    tio.RandomAffine(scales=(0.9, 1.1), degrees=10),  # Random scaling and rotation
    tio.RandomElasticDeformation(),  # Random elastic deformation
])

# Assuming img_dir and label_dir are defined
dataset = imagedata(img_dir='/Volumes/toby/BRATS2017/TrainingDataset/images', label_dir='/Volumes/toby/BRATS2017/TrainingDataset/labels', transform=data_transforms)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True, num_workers=2)

# Iterate over the DataLoader in the training loop
for epoch in range(1):
    for batch_idx, (images, masks) in enumerate(dataloader):
        t1_images = images['t1']
        print(f"Epoch {epoch}, Batch {batch_idx}, T1 Image Shape: {t1_images.shape}")
        print(f"Mask Shape: {masks.shape}")


Exception ignored in: <bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x7ff09be73c50>>
Traceback (most recent call last):
  File "/Users/tobymao/anaconda3/envs/ipk/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 1328, in __del__
    self._shutdown_workers()
  File "/Users/tobymao/anaconda3/envs/ipk/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 1320, in _shutdown_workers
    if w.is_alive():
  File "/Users/tobymao/anaconda3/envs/ipk/lib/python3.6/multiprocessing/process.py", line 134, in is_alive
    Exception ignored in: assert self._parent_pid == os.getpid(), 'can only test a child process'<bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x7ff09be73c50>>

AssertionErrorTraceback (most recent call last):
:   File "/Users/tobymao/anaconda3/envs/ipk/lib/python3.6/site-packages/torch/utils/data/data

Epoch 0, Batch 0, T1 Image Shape: torch.Size([4, 1, 128, 128, 128])
Mask Shape: torch.Size([4, 1, 128, 128, 128])
Epoch 0, Batch 1, T1 Image Shape: torch.Size([4, 1, 128, 128, 128])
Mask Shape: torch.Size([4, 1, 128, 128, 128])
Epoch 0, Batch 2, T1 Image Shape: torch.Size([4, 1, 128, 128, 128])
Mask Shape: torch.Size([4, 1, 128, 128, 128])
Epoch 0, Batch 3, T1 Image Shape: torch.Size([4, 1, 128, 128, 128])
Mask Shape: torch.Size([4, 1, 128, 128, 128])


KeyboardInterrupt: 

In [12]:
#check and see if the dataloader function works
dataset = imagedata(img_dir='/Volumes/toby/BRATS2017/TrainingDataset/images', label_dir='/Volumes/toby/BRATS2017/TrainingDataset/labels')
index = 0 
image,mask = dataset.__getitem__(index)


print(image['t1'][0].shape)
print(len(mask))
print((torch.from_numpy(image['t1'])).shape)
print(nib.load('/Volumes/toby/BRATS2017/TrainingDataset/images/Brats17_2013_0_1_flair.nii').shape)

(240, 155)
240
torch.Size([240, 240, 155])
(240, 240, 155)


In [6]:
from torchvision import transforms
class ToTensorAndNormalize:
    def __call__(self, sample):
        image, mask = sample['image'], sample['mask']
        
        # Convert to tensor
        t1 = torch.tensor(image['t1'], dtype=torch.float32)
        flair = torch.tensor(image['flair'], dtype=torch.float32)
        t2 = torch.tensor(image['t2'], dtype=torch.float32)
        mask = torch.tensor(mask, dtype=torch.float32)

        # Normalize images
        t1 = (t1 - t1.mean()) / t1.std()
        flair = (flair - flair.mean()) / flair.std()
        t2 = (t2 - t2.mean()) / t2.std()

        return {
            'image': {
                't1': t1,
                'flair': flair,
                't2': t2
            },
            'mask': mask
        }


In [8]:
# Instantiate the dataset with the custom transform
transform = transforms.Compose([ToTensorAndNormalize()])

dataset = imagedata(img_dir='/Volumes/toby/BRATS2017/TrainingDataset/images', label_dir='/Volumes/toby/BRATS2017/TrainingDataset/labels')
index = 0  
image, mask = dataset[index]

print(image['t1'].shape)

(240, 240, 155)
